In [2]:
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests

options = webdriver.ChromeOptions()
options.set_capability('goog:loggingPrefs', {"performance": "ALL", "browser": "ALL"})
# Adding user agent to look more like a real browser
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.set_page_load_timeout(30)

In [11]:
def get_score(match_id, driver):
    # Clear existing logs
    #driver.get("about:blank")
    #logs = driver.get_log("performance")
    
    # Load the page
    """
    print(f"Loading URL: {url}")
    driver.get(url)
    """
    
    """
    # Wait for the page to load properly
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )
    except:
        print("Timeout waiting for page to load")
    """
    """
    # Interact with the page to trigger API calls
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(0.1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.2)

    soup = BeautifulSoup (driver.page_source, "html.parser")

    meta_data = soup.find_all('script', type = 'application/ld+json')

    meta_data = json.loads(meta_data[1].string)
    """
    # Direct API approach - try to call the API directly
    
    if match_id:
        print(f"Extracted match ID: {match_id}")
        urls = []
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}/average-positions")
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}/lineups")

        
        data = {}
        for i in range(0,len(urls)):
            api_url = urls[i]
            # Open the API URL directly
            #driver.execute_script(f"window.open('{api_url}', '_blank');")

            driver.get(api_url)
            #time.sleep(2)
            
            # Switch to the new tab
            #driver.switch_to.window(driver.window_handles[-1])
            #time.sleep(1)
            
            # Get the page source which should contain the JSON
            page_source = driver.page_source
            
            # Extract JSON from the page source
            if "application/json" in page_source or "{" in page_source:
                start_idx = page_source.find("{")
                end_idx = page_source.rfind("}") + 1
                if start_idx >= 0 and end_idx > start_idx:
                    json_str = page_source[start_idx:end_idx]
                    try:
                        data[i] = json.loads(json_str)
                    except json.JSONDecodeError:
                        print("Failed to parse JSON response")
            
            # Close the tab and switch back
            #driver.close()
            #driver.switch_to.window(driver.window_handles[0])

        #data[len(urls)] = meta_data
        return data

In [6]:
data = (get_score(url,'12437027',driver))

Extracted match ID: 12437027


In [5]:
url = "https://www.sofascore.com/football/match/manchester-united-leicester-city/GK#id:12437027,tab:statistics"

In [19]:
print (data[0])

{'home': [{'player': {'name': 'Stephy Mavididi', 'firstName': '', 'lastName': '', 'slug': 'stephy-mavididi', 'shortName': 'S. Mavididi', 'position': 'M', 'jerseyNumber': '10', 'userCount': 1567, 'id': 605338, 'fieldTranslations': {'nameTranslation': {'ar': 'ستيفي مافيديدي', 'hi': 'स्टेफी माविडिडी', 'bn': 'স্টেফি মাভিদিদি'}, 'shortNameTranslation': {'ar': 'س. مافيديدي', 'hi': 'एस. माविडिडी', 'bn': 'এস. মাভিদিদি'}}}, 'averageX': 56.114285714286, 'averageY': 70.852380952381, 'pointsCount': 21}, {'player': {'name': 'Jordan Ayew', 'firstName': '', 'lastName': '', 'slug': 'jordan-ayew', 'shortName': 'J. Ayew', 'position': 'F', 'jerseyNumber': '18', 'userCount': 18273, 'id': 103045, 'fieldTranslations': {'nameTranslation': {'ar': 'جوردان أيو', 'hi': 'जॉर्डन अय्यू', 'bn': 'জর্ডান আইউ'}, 'shortNameTranslation': {'ar': 'ج. أيو', 'hi': 'जे. अय्यू', 'bn': 'জে. আইউ'}}}, 'averageX': 71.353333333333, 'averageY': 56.38, 'pointsCount': 15}, {'player': {'name': 'Jamie Vardy', 'slug': 'jamie-vardy', 'sho

In [28]:
## average positions = 0 
## lineups = 1
def parse_data (data):
    ## get name of player:
    players = []
    
    player_stats = ['totalPass','accuratePass','totalLongBalls','goalAssist','duelLost','duelWon','dispossessed','totalContest','wonContest','errorLeadingToAShot','wasFouled','minutesPlayed','touches','rating','expetedGoals','keyPass','goals','bigChanceCreated','totalOffside','totalTackle']
    lineups = data[1]
    positions = data[0]
    for cat in ['home','away']:
        for player in lineups[cat]['players']:
            if player['substitute'] == False:
                player_data = {}
                player_data['name'] = player['player']['slug']
                player_data['position'] = player['player']['position']
                for x in player_stats: 
                    if x in player['statistics']:
                        player_data[x] = str(player['statistics'][x])
                    else:
                        player_data[x] = '0'
                for entry in positions[cat]:
                    if entry['player']['slug'] == player_data['name']:
                        player_data['positionX'] = str(entry['averageX'])
                        player_data['positionY'] = str(entry['averageY'])
                if 'positionX' not in player_data:
                    player_data['postionX'] = '-1'
                    player_data['positionY'] = '-1'
                player_data['home'] = "1" if cat == "home" else "0"
                #print (player_data)
                players.append(player_data)
    return (players)
        
    

In [21]:
print(parse_data(data))

[{'name': 'mads-hermansen', 'position': 'G', 'totalPass': 33, 'accuratePass': 24, 'totalLongBalls': 15, 'goalAssist': 0, 'duelLost': '0', 'duelWon': '0', 'dispossessed': '0', 'totalContest': '0', 'wonContest': '0', 'errorLeadingToAShot': '0', 'wasFouled': '0', 'minutesPlayed': 90, 'touches': 41, 'rating': 6.1, 'expetedGoals': '0', 'keyPass': '0', 'goals': '0', 'bigChanceCreated': '0', 'totalOffside': '0', 'totalTackle': '0', 'positionX': 13.802127659574, 'positionY': 50.223404255319, 'home': '1'}, {'name': 'wout-faes', 'position': 'D', 'totalPass': 73, 'accuratePass': 63, 'totalLongBalls': 2, 'goalAssist': 0, 'duelLost': 4, 'duelWon': 5, 'dispossessed': '0', 'totalContest': '0', 'wonContest': '0', 'errorLeadingToAShot': '0', 'wasFouled': 1, 'minutesPlayed': 90, 'touches': 91, 'rating': 6.9, 'expetedGoals': '0', 'keyPass': 1, 'goals': '0', 'bigChanceCreated': '0', 'totalOffside': '0', 'totalTackle': 2, 'positionX': 42.618095238095, 'positionY': 26.757142857143, 'home': '1'}, {'name': 'c

In [25]:
def get_game_name(match_id,driver):
    driver.get(f"https://www.sofascore.com/api/v1/event/{match_id}")
    #time.sleep(2)
    
    # Switch to the new tab
    #driver.switch_to.window(driver.window_handles[-1])
    #time.sleep(1)
    
    # Get the page source which should contain the JSON
    page_source = driver.page_source
    
    # Extract JSON from the page source
    if "application/json" in page_source or "{" in page_source:
        start_idx = page_source.find("{")
        end_idx = page_source.rfind("}") + 1
        if start_idx >= 0 and end_idx > start_idx:
            json_str = page_source[start_idx:end_idx]
            try:
                data = json.loads(json_str)
            except json.JSONDecodeError:
                print("Failed to parse JSON response")
    home_team = data['event']['homeTeam']['slug']
    away_team = data['event']['awayTeam']['slug']
    return home_team + '_vs_' + away_team
    

In [29]:
file = open("match_links_24_25.txt", "r")
for match_id in file:
    match_id = match_id.strip()
    data = get_score(match_id,driver)
    player_data = parse_data(data)
    file_name = get_game_name(match_id,driver)
    with open(f"player_data/24_25/{file_name}.csv", "a") as f:
        for player in player_data:
            f.write (",".join(player.values()) + "\n")
        

f.close()

Extracted match ID: 12436965
Extracted match ID: 12436914
Extracted match ID: 12436629
Extracted match ID: 12436621
Extracted match ID: 12436909
Extracted match ID: 12436980
Extracted match ID: 12436500
Extracted match ID: 12436981
Extracted match ID: 12436955
Extracted match ID: 12436883
Extracted match ID: 12436983
Extracted match ID: 12436905
Extracted match ID: 12436504
Extracted match ID: 12436523
Extracted match ID: 12436496
Extracted match ID: 12436906
Extracted match ID: 12436623
Extracted match ID: 12436514
Extracted match ID: 12436991
Extracted match ID: 12436478
Extracted match ID: 12436460
Extracted match ID: 12436885
Extracted match ID: 12436463
Extracted match ID: 12437046
Extracted match ID: 12436932
Extracted match ID: 12436982
Extracted match ID: 12436472
Extracted match ID: 12436944
Extracted match ID: 12437027
Extracted match ID: 12436871
Extracted match ID: 12436458
Extracted match ID: 12436624
Extracted match ID: 12436887
Extracted match ID: 12436989
Extracted matc